# View CP2K Calculation Results

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.calculation.job.cp2k import Cp2kCalculation
from aiida.orm.code import Code

from aiida.orm import load_node 

import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview

import urlparse
from collections import OrderedDict

In [ ]:
%%html
<!-- fix width of descriptions -->
<style>
.widget-hbox .widget-label {width: 150px; max-width: 150px}
</style>

In [ ]:
def get_all_calcs():
    # query AiiDA database for structures
    qb = QueryBuilder()
    qb.append(Cp2kCalculation, project=["id", "description"])
    #qb.order_by({StructureData: {'ctime': 'desc'}})
    #print qb.count()
    qb.all()

    all_calcs = OrderedDict()
    for pk, descr in qb.iterall():
        label = "PK: %d"%pk
        if descr:
            label += "; descr: "+descr
        all_calcs[label] = pk
    return all_calcs

In [ ]:
def reload_calcs(b):
    inp_calc.options = get_all_calcs()
    update_view(None)
    
#inp_calc = widgets.Dropdown(description="Calculation", options=get_all_calcs())
inp_calc = widgets.Select(description="Calculation", options=get_all_calcs())
btn_reload_cals = widgets.Button(description="Reload")
btn_reload_cals.on_click(reload_calcs)

# parse URL
url = urlparse.urlsplit(jupyter_notebook_url)
querypart = urlparse.parse_qs(url.query)
if 'pk' in querypart.keys():
    inp_calc.value = int(querypart['pk'][0])

display(widgets.HBox([inp_calc, btn_reload_cals]))

In [ ]:
info_box = widgets.HTML()

def update_view(c):
    clear_output()
    if inp_calc.value == None:
        return
    node = load_node(inp_calc.value)
    output = str(node) + "\n"
    output += "state: "+node.get_state() +"\n"
    if node.get_state() == "FINISHED":
        output += "Energy: %e %s\n"%(node.res.energy, node.res.energy_units)
    info_box.value = "<pre>"+output+"</pre>"

update_view(None)
inp_calc.observe(update_view, names='value')

display(info_box)